In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt 

from tslearn.clustering import TimeSeriesKMeans, KernelKMeans
from tslearn.metrics import dtw

from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
from tslearn.datasets import CachedDatasets

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
import seaborn as sbn

import networkx as nx

from sklearn.svm import SVC
from sklearn.feature_selection import RFE

import os

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets

ModuleNotFoundError: No module named 'tslearn'

In [ ]:
os.listdir('../data/HER2/')

In [ ]:
load = 'normalized' # or 'raw'

series_sel = pd.read_csv('../data/HER2/H210122_SKBR3/normalized/clover_all_cell.csv').columns[1:-3]

_datas = []
for dataset in os.listdir('../data/HER2'): 
    cl_path = '../data/HER2/' + dataset + '/' + load + '/clover_all_cell.csv'
    ms_path = '../data/HER2/' + dataset + '/' + load + '/mscarlet_all_cell.csv'
    _clover = pd.read_csv(cl_path)
    _mscarl = pd.read_csv(ms_path)
    _data = _clover.merge(_mscarl, on=['track_index', 'cell__treatment'], how='inner')
    _data = _data.assign(dataset=dataset)
    _datas.append(_data)
    
data = pd.concat(_datas, axis=0)

clover_sel = [f'{x}_x' for x in series_sel]
mscarl_sel = [f'{x}_y' for x in series_sel]

data = data.assign(drug = [x.split('_', maxsplit=5)[-1] for x in data.cell__treatment])
data = data.assign(cell_line = [x.split('_', maxsplit=5)[0] for x in data.cell__treatment])
data = data.assign(mutant = [x.split('_', maxsplit=5)[-2] for x in data.cell__treatment])

data.head()

In [ ]:
plt.figure(figsize=(35, 5))
plt.bar(x=data.columns, height=data.isna().sum())
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.drug.unique()

In [ ]:
data = data[lambda x: x.drug.isin(['untreated', '10ug_ml_trastuzumab'])]
data.shape

In [ ]:
data.drug.unique()

In [ ]:
plt.figure(figsize=(35, 5))
plt.bar(x=data.columns, height=data.isna().sum())
plt.xticks(rotation=90)
plt.show()

In [ ]:
# remove na
clover_sel = np.array(clover_sel)[~data[clover_sel].isna().any()]
mscarl_sel = np.array(mscarl_sel)[~data[mscarl_sel].isna().any()]

In [ ]:
#X_train = data[clover_sel]
X_train = np.stack([data[clover_sel], data[mscarl_sel]], axis=2)
print(X_train.shape)

# Make time series shorter
X_train = TimeSeriesResampler(sz=100).fit_transform(X_train)
sz = X_train.shape[1]
print(X_train.shape)
nclus = 25

In [ ]:
km = TimeSeriesKMeans(n_clusters=nclus, verbose=True, random_state=0, metric='euclidean', n_jobs=8)
y_pred = km.fit_predict(X_train)

In [ ]:
plt.figure(figsize=(20,10))
for yi in range(nclus):
    plt.subplot(5, 5, yi + 1)
    for xx in X_train[y_pred == yi][0:250]:
        plt.plot(xx[:,0], "r-", alpha=.05)
        plt.plot(xx[:,1], "b-", alpha=.05)
        
    plt.title(f'cluster sz: {len(X_train[y_pred == yi])}')
    plt.plot(km.cluster_centers_[yi][:,0], "r-", label='clover')
    plt.plot(km.cluster_centers_[yi][:,1], "b-", label='mscarlet')
    
    plt.xlim(0, sz)
    plt.ylim(0, 1)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)

plt.tight_layout()
plt.show()

In [ ]:
lb = LabelEncoder()
y_trt = lb.fit_transform([f'{x}--{y}' for x,y in zip(data.cell__treatment.values, data.dataset.values)])

In [ ]:
cm_cnts = {c:np.zeros(nclus) for c in lb.classes_} 

In [ ]:
for i, clus, grp in zip(range(len(y_pred)), y_pred, y_trt) :
    cm_cnts[lb.classes_[grp]][clus] += 1
    
cm_prob = {k:v/np.sum(v) for k,v in cm_cnts.items()}

In [ ]:
labels = [k for k,v in cm_prob.items()]
cm = np.stack([v for k,v in cm_prob.items()], axis=0)

In [ ]:
cm.shape

In [ ]:
corr = np.corrcoef(cm, rowvar=False)

plt.figure(figsize=(7,7))
ax = sbn.clustermap(
    corr, 
    vmin=-1, vmax=1, center=0,
    square=True
)
plt.show()

In [ ]:
cm_stds = cm.std(axis=0)

plt.figure()
plt.hist(cm_stds, bins=7)
plt.ylabel('count')
plt.xlabel('standard deviation')
plt.show()

In [ ]:
#drug = dict(zip(data.drug.unique(), "rbg"))
#row_colors = data.drug.map(drug)
#g = sbn.clustermap(cm, row_colors=row_colors)
# [x.split('--')[0].split('_')[-1] for x in lb.classes_], 'cell_line':[x.split('_')[4] for x in lb.classes_]

drug = [x.split('--')[0].split('_')[-1] for x in labels]
lut = dict(zip(set(drug), sbn.hls_palette(len(set(drug)), l=0.5, s=0.8)))
row_colors = pd.DataFrame(drug)[0].map(lut)

#Create additional row_colors here
cell_line = [x.split('_')[4] for x in labels]
lut2 = dict(zip(set(cell_line), sbn.hls_palette(len(set(cell_line)), l=0.5, s=0.8)))
row_colors2 = pd.DataFrame(cell_line)[0].map(lut2)

df = pd.DataFrame(index=labels, data=cm)
sbn.clustermap(df, figsize=(12,15), row_colors=[row_colors, row_colors2]) 

#plt.ylabel('treatment')
#plt.yticks(ticks=plt.yticks()[0], labels=[str(x) for x in labels])
#plt.xlabel('cluster label')
plt.show()

# Dimensionality Reduction

In [ ]:
pca = PCA(n_components=2)
PCs = pca.fit_transform(cm)

print('explained variance ratio:', pca.explained_variance_ratio_)
print('PC shape:', PCs.shape)
res = pd.DataFrame({'pc1': PCs[:,0], 'pc2':PCs[:,1], 'treatment':[x.split('--')[0].split('_')[-1] for x in lb.classes_], 'cell_line':[x.split('_')[4] for x in lb.classes_]})
res.head()

In [ ]:
plt.figure(figsize=(10,13))
sbn.scatterplot(x='pc1', y='pc2', data=res, hue='cell_line', style='treatment', s=300)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
sbn.scatterplot(x='pc1', y='pc2', data=res[lambda x: (x.cell_line.isin(['WT', 'nd611']))], hue='cell_line', style='treatment', s=300)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
res.head()

In [ ]:
res_ner = res[lambda x: (x.cell_line.isin(['WT', 'T798I']))]# & (x.treatment == 'neratinib')]

In [ ]:
X = 5* res_ner[['pc1', 'pc2']].values
y = 1.*((res_ner.cell_line == 'WT') & (res_ner.treatment == '10ug_ml_trastuzumab')).values \
        + 2.* (res_ner.treatment == 'untreated').values

In [ ]:
np.unique(y)

In [ ]:
n_features = X.shape[1]

C = 10
kernel = 1.0 * RBF([1.0, 1.0])  # for GPC

# Create different classifiers.
classifiers = {
    'L1 logistic': LogisticRegression(C=C, penalty='l1',
                                      solver='saga',
                                      multi_class='multinomial',
                                      max_iter=10000),
    'L2 logistic (Multinomial)': LogisticRegression(C=C, penalty='l2',
                                                    solver='saga',
                                                    multi_class='multinomial',
                                                    max_iter=10000),
    'L2 logistic (OvR)': LogisticRegression(C=C, penalty='l2',
                                            solver='saga',
                                            multi_class='ovr',
                                            max_iter=10000),
    'Linear SVC': SVC(kernel='linear', C=C, probability=True,
                      random_state=0),
    'GPC': GaussianProcessClassifier(kernel)
}

n_classifiers = len(classifiers)

plt.figure(figsize=(3 * 3, n_classifiers * 3))
plt.subplots_adjust(bottom=.2, top=.95)

xx = np.linspace(-1, 1, 100)
yy = np.linspace(-1, 1, 100).T
xx, yy = np.meshgrid(xx, yy)
Xfull = np.c_[xx.ravel(), yy.ravel()]

class_names = ['resistant', 'sensitive', 'untreated']
for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X, y)

    y_pred = classifier.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))

    # View probabilities:
    probas = classifier.predict_proba(Xfull)
    n_classes = np.unique(y_pred).size
    for k in range(n_classes):
        plt.subplot(n_classifiers, n_classes, index * n_classes + k + 1)
        plt.title("%s class" % class_names[k])
        if k == 0:
            plt.ylabel(name)
        imshow_handle = plt.imshow(probas[:, k].reshape((100, 100)),
                                   extent=(-1, 1, -1, 1), origin='lower')
        plt.xticks(())
        plt.yticks(())
        idx = (y_pred == k)
        if idx.any():
            plt.scatter(X[idx, 0], X[idx, 1], marker='o', c='w', edgecolor='k')

ax = plt.axes([0.15, 0.04, 0.7, 0.05])
plt.title("Probability")
plt.colorbar(imshow_handle, cax=ax, orientation='horizontal')

plt.show()

In [ ]:
X_all = 5*res[['pc1', 'pc2']].values

In [ ]:
y_hat = classifiers['Linear SVC'].predict_proba(X_all)

In [ ]:
f, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].hist(y_hat[:,0])
ax[0].set_title('resistant')
ax[1].hist(y_hat[:,1])
ax[1].set_title('sensitive')
ax[2].hist(y_hat[:,2])
ax[2].set_title('untreated')
plt.show()

In [ ]:
pres = pd.DataFrame({'prob_res':y_hat[:,0], 'prob_sens':y_hat[:,1], 'prob_untreat':y_hat[:,2]})
res2 = pd.concat([res, pres], axis=1)
res2 = res2.assign(call=[['res','sens','untreat'][np.argmax([x,y,z])] for x,y,z in zip(res2.prob_res, res2.prob_sens, res2.prob_untreat)])
res2.head()

In [ ]:
res2.groupby(['call', 'treatment'])[['cell_line']].count()

In [ ]:
res2[lambda x: x.prob_res > 0.6].sort_values('prob_res', ascending=False)

In [ ]:
res2[lambda x: x.prob_sens > 0.8].sort_values('prob_sens', ascending=False)

In [ ]:
res2[lambda x: x.prob_untreat > 0.75]